In [1]:
import numpy as np
import pandas as pd

from arch import arch_model
from arch.univariate import EGARCH
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

# local module
from data_cleaning.import_data import import_data
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

In [17]:
# estimate and report parameter estimates and misspec tests for all pairs
pairs = {'USDRUB':'2005-03-03', 'USDZAR':'2003-02-23', 'USDBRL':'2003-10-01', 'EURUSD':'2000-03-15', 'USDJPY':'2000-02-23', 'GBPUSD':'2000-02-23'}
#params = pd.DataFrame(index=['omega', 'alpha', 'beta', 'nu', 'aic', 'logL'])
#tvals = pd.DataFrame(index=['omega', 'alpha', 'beta', 'nu'])
params = pd.DataFrame(index=['omega', 'alpha', 'beta', 'gamma', 'nu', 'aic', 'logL'])
tvals = pd.DataFrame(index=['omega', 'alpha', 'beta', 'gamma', 'nu'])
ljung_box = pd.DataFrame()
ljung_box.index += 1 
noarch = pd.DataFrame(index=['LM statistic', 'Degrees of Freedom', 'P-value'])

for pair in pairs:
    df = import_data(fx_pair=pair)
    ret = df['log_ret']
    ret = ret.loc[pairs[pair]:'2020-12-31'] * 100
    #ret[0] = 0

    # fit model and save residuals
    #garch = arch_model(ret, mean='Zero', vol='GARCH', p=1,q=1, dist = 'Studentst')
    garch = arch_model(ret, mean='Zero', vol='GARCH', p=1,q=1,o=1, dist = 'Studentst')
    garch_fit = garch.fit(update_freq = 0, disp = 'off')
    res = garch_fit.resid
    
    # fetch params
    params_it = []
    params_it.append(garch_fit.params['omega'])
    params_it.append(garch_fit.params['alpha[1]'])
    params_it.append(garch_fit.params['beta[1]'])
    params_it.append(garch_fit.params['gamma[1]'])
    params_it.append(garch_fit.params['nu'])
    params_it.append(garch_fit.aic)
    params_it.append(garch_fit.loglikelihood)
    params[pair] = params_it

    # fetch t-values
    tvals_it = []
    tvals_it.append(garch_fit.tvalues['omega'])
    tvals_it.append(garch_fit.tvalues['alpha[1]'])
    tvals_it.append(garch_fit.tvalues['beta[1]'])
    tvals_it.append(garch_fit.tvalues['gamma[1]'])
    tvals_it.append(garch_fit.tvalues['nu'])
    tvals[pair] = tvals_it

    # misspecification tests
    lb = sm.stats.acorr_ljungbox(res, lags=list(range(1,11)), return_df=True) # Ljung-Box test. H0: No residual autocorrelation
    ljung_box[pair] = lb['lb_pvalue']

    noarch_test = garch_fit.arch_lm_test(5, True) # Engle no ARCH test. H0: No ARCH effects in residuals
    noarch_it = []
    noarch_it.append(noarch_test.stat)
    noarch_it.append(noarch_test.df)
    noarch_it.append(noarch_test.pval)
    noarch[pair] = noarch_it


/var/folders/sh/xyhn9mnd0fldcy0brzhtlv280000gn/T/ipykernel_80241/915748668.py:14: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  ret = ret.loc[pairs[pair]:'2020-12-31'] * 100


In [18]:
params

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
omega,0.000879,0.019598,0.010806,0.000735,0.003678,0.002684
alpha,0.085573,0.074591,0.133847,0.029951,0.049019,0.029768
beta,0.940310,0.928485,0.900406,0.963295,0.933272,0.951983
gamma,-0.051765,-0.044275,-0.085214,0.010625,0.020074,0.020265
nu,5.716412,10.465738,8.270226,10.151902,5.493424,8.682367
aic,7114.360706,13251.646747,11413.015682,8902.573665,8936.997572,8523.559120
logL,-3552.180353,-6620.823374,-5701.507841,-4446.286832,-4463.498786,-4256.779560


In [19]:
tvals

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
omega,3.117350,3.075954,3.538415,2.011089,2.944750,2.591342
alpha,6.504477,5.722930,8.063578,5.910223,6.705394,4.091427
beta,94.098644,68.294065,74.742777,238.512875,95.880578,109.141924
gamma,-4.996624,-3.674408,-5.704937,1.725039,1.812921,2.416105
nu,11.470940,6.602846,7.841849,7.861989,13.541812,7.610028


In [20]:
ljung_box.round(3)

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
1,0.484,0.309,0.0,0.370,0.006,0.012
2,0.059,0.257,0.0,0.654,0.023,0.041
3,0.038,0.278,0.0,0.615,0.056,0.093
4,0.060,0.399,0.0,0.652,0.052,0.170
5,0.073,0.528,0.0,0.439,0.090,0.039
6,0.048,0.411,0.0,0.515,0.124,0.058
7,0.080,0.512,0.0,0.606,0.149,0.091
8,0.090,0.302,0.0,0.682,0.160,0.114
9,0.000,0.392,0.0,0.375,0.223,0.120
10,0.000,0.074,0.0,0.392,0.139,0.038


In [15]:
noarch.round(3)

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
LM statistic,24.962,9.390,10.651,7.180,2.089,14.933
Degrees of Freedom,5.000,5.000,5.000,5.000,5.000,5.000
P-value,0.000,0.094,0.059,0.208,0.837,0.011
